In [ ]:
!pip install pandas --quiet
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install duckdb --quiet
!pip install jupysql --quiet
!pip install duckdb-engine --quiet
!pip install folium matplotlib mapclassify --quiet

In [ ]:
import pandas as pd
import geopandas as gpd
import duckdb
from shapely import wkt
from shapely import Point
from shapely.ops import nearest_points
import shapely

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb://

In [ ]:
%sql INSTALL spatial;
%sql INSTALL httpfs;
%sql LOAD spatial;
%sql LOAD httpfs;
%sql SET s3_region='us-west-2'

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
merged_data = pd.read_parquet(path='merged_data.parquet')

In [ ]:
convert_columns = ['original_coord', 'snapped_coord', 'osm_coord']
for col in convert_columns:
    merged_data[col] = merged_data[col].apply(wkt.loads)

In [ ]:
all_snap = gpd.GeoDataFrame(merged_data, geometry='snapped_coord', crs="EPSG:4326")

In [ ]:
selective_snap = all_snap.copy(deep=True)

In [ ]:
categories_to_exclude = ['farm', 'lake', 'park', 'field', 'campground', 'river']
selective_snap['snapped_coord'] = selective_snap.apply(lambda x: x['original_coord'] if any(cat in x['category'] for cat in categories_to_exclude) else x['snapped_coord'], axis=1)
selective_snap['new_dist'] = selective_snap.apply(lambda x: x['old_dist'] if any(cat in x['category'] for cat in categories_to_exclude) else x['new_dist'], axis=1)

In [ ]:
all_snap['diff'] = all_snap['new_dist'] - all_snap['old_dist']
selective_snap['diff'] = selective_snap['new_dist'] - selective_snap['old_dist']

In [ ]:
print(all_snap['diff'].mean())
print(selective_snap['diff'].mean())